# 贝叶斯决策论和朴素贝叶斯分类器

徐迪深 浙江工商大学 金融学院 CFA1901

## 贝叶斯决策论

在炎热的夏天你想去市场上挑选西瓜。市场里有一堆西瓜，但是你只想要又甜肉又多的西瓜。那么这个时候你遇到的就是一个分类问题：你需要将面前的瓜分成类：又甜肉又多的西瓜和其它西瓜。那么这个时候你该用什么方法将西瓜分类呢？

面对分类问题，贝叶斯决策论提供了一个通过计算概率来对样本进行分类的方法。

### 条件风险

你挑选西瓜的时候是完全有可能犯错的，贝叶斯决策论的思想把你犯错的风险最小化。衡量风险的指标有很多，在贝叶斯决策论中最常用的就是条件风险(conditional risk)。

假设有N种可能的类别标记，即$Y=${$c_1,c_2,...,c_N$}，$\lambda_{ij}$是将一个真实类别为$c_j$的样本误分为$c_i$所产生的损失，那么基于条件概率$P(c_i|x)$(可以理解为当你有一个不知道类别的样本x，你有多大的概率会将x分为$c_i$这一类)，那么条件风险我们用如下的式子表示：

$$
R(c_i|x)=\sum_{j=1}^N\lambda_{ij}P(c_i|x)\qquad(1)
$$

比如在刚才的例子中，设可能的类别标签$Y=${$1,2,3,4$}，分别表示甜而肉多，甜而肉少，不甜肉多，不甜肉少。我们假设市场上有10个西瓜，它们的真实类别$X=${$1,2,3,2,4,2,3,4,2,1$}，而我们实际分类是{$1,3,3,4,3,2,3,2,2,1$}在python程序我们可以用列表来表示这两个集合

In [1]:
Y = [1,2,3,4]#类别
X = [1,2,3,2,4,2,3,4,2,1]#实际分类
X_class = [1,2,3,4,3,2,3,2,2,1]#X_class指的是你对样本做出的分类
#这样我们就能写出一个程序判断分类的对错，我们程序输入样本真实类别和我们分出的类别
#输出的结果是一个列表，列表的值是0或1，1表你的分类是与实际不符，0表示分类与实际相符
def compare(X,X_class):
    result = []
    for i in range(len(X_class)):
        if X_class[i] == X[i]:
            result.append(0)
        else:
            result.append(1)
    return result
compare_result = compare(X,X_class)
compare_result

[0, 0, 0, 1, 1, 0, 0, 1, 0, 0]

同时我们假设：$P(1|x)=0.2$，$P(2|x)=0.3$，$P(3|x)=0.3$，$P(4|x)=0.2$

而我们的分类错误的损失也可以用上述的方法表示，假设$\lambda_{ij}=1(i\neq j;i,j=1,2,3,4)$.

In [94]:
lambda_ij = 1#分类错误所产生的损失
#同时我们还需要知道，我们错误的分类是被错分成了哪一类
def analysis(X_class,compare_result,P_ix):
    def multiply(x,y):
        return x*y
    index = list(map(multiply,compare_result,X_class))
    return index
analysis_result = analysis(X_class,compare_result,P_ix)
analysis_result

[0, 0, 0, 4, 3, 0, 0, 2, 0, 0]

那么根据上面的条件风险的计算公式，有

$$
Risk=\lambda_{42}P(4|x)+\lambda_{34}P(3|x)+\lambda_{24}P(2|x)
$$

In [3]:
Risk = 1*0.2+1*0.3+1*0.3
Risk

0.8

### 条件风险的最小化

很显然，条件风险是最小最好的。计算条件风险时我们将分类错误的损失和分类错误的概率匹配以后计算期望值。在分类的任务中我们依据场景有不同的误分损失，但是只要确定好具体任务背景误分损失就是确定的。所以我们如果想要将条件风险最小就是要将误分的概率最小即可。由概率之和为1的公理我们又得出：最小化分类错误的条件概率等价于最大化正确分类的条件概率。

回到上面的例子。

我们想要得到的是甜且薄皮的西瓜（第一类西瓜），那么我们就想要最大化$P(1|x)$.由贝叶斯公式

$$
P(1|x)=P(1)\frac{P(x|1)}{P(x)}\qquad(2)
$$

很显然，当我们来到一个瓜摊，西瓜的数量是固定的，所以上上述四类西瓜的个数也分别是固定的。假如这个瓜摊里面有足够多的西瓜，那么根据大数定理：概率可以通过频率无限逼近得到. 那么$(2)$当中的$P(1)$(以及其它三类的概率)就是固定的。这个类别的概率我们也称为先验（prior）概率。而同理，$P(x)$(每一种样本的概率)也应该是固定的。

显然，如果我们想要最大化正确分类的条件概率，那么就应该最大化$P(x|1)$。这个概率也称为似然（likelihood）概率。$P(x|1)$的理解其实并不难，实际上这个条件概率就是在$P(x)$的基础上加了是在类别1的西瓜里进行了统计的条件。所以$(2)$也给了我们一个启示：

$$
\frac{P(x|1)}{P(x)}\qquad(3)
$$

假如$(3)$能够大于1，就意味着比起我们的总体样本，拥有x的特征的西瓜更容易在1类别的样本中被找到，那么结合之前我们的先验概率，在一堆特征都是x的样本中属于1类别的样本出现的可能更大。

### 极大似然估计

回到上面一节我们的思路：在一个分类任务下我们希望能最小化我们的分类风险，于是我们就运用概率的手段构造了条件风险，希望能够找到条件风险最小的一个方法。通过条件风险的表达式，我们发现，最小化条件风险等价于最小化错误分类的概率等价于最大化正确分类的概率。所以我们运用贝叶斯公式将条件概率拆分发现最大化正确分类的概率等价于最大化似然概率。

现在的问题是如何最大化似然概率。我们常用的方法就是极大似然估计。

极大似然估计是参数估计中常用的方法。假设我们的样本服从同一个已知的分布，那么想要估计这些样本服从的概率密度函数，唯一要估计的就是一直分布的参数。在随机抽样的情况下，参数(假设就是正态分布)$\mu_S,\sigma_S$样本空间$S$={$X_1,X_2,...,X_n$}的似然就是

$$
P(S|\mu_S,\sigma_S)=\prod_{X_n\in S}P(X_n|\mu_S,\sigma_S)\qquad(4)
$$

通过对$(4)$取对数然后再对参数求导就可以找到使样本似然概率最大的值。实际上，极大似然估计就是想在所有参数中找到能使样本出现的可能性最大的值，或者说最符合我们样本的概率密度函数。

## 朴素贝叶斯分类器

贝叶斯分类器有很多种，而朴素贝叶斯分类器是在所有特征都独立的假设进行分类的一种算法。

对于我们的样本，我们需要用特征来区分样本。比如一个西瓜的色泽，瓜的形状都是对于西瓜这个样本的特征。假设我们的特征集合$F$={$f_1,f_2,f_3,...,f_i,...,f_n$}，我们想要分出的类别集合$C$={$c_1,c_2,c_3,...,c_i,...,c_n$},在独立的假设下，这样$(2)$就可以写成如下式子

$$
P(c_i|f_i)=\prod_{f_i\in F}P(f_i|c_i)\frac{P(c_i)}{P(x)}\qquad(5)
$$

可以看出来，在独立的假设下，原本困难的所有特征和类别联合分布的估计变的简单很多。这也是朴素贝叶斯分类器的优势：能够在比较有限的样本中估计，但相对的也会损失估计的精度。



### 分类的判断标准

贝叶斯决策论更像是从类别到特正的判断，就如上面购买西瓜的例子：寻找甜而且皮薄的西瓜更有可能具有的特征，然后挑选更符合这些特征的西瓜。然而贝叶斯分类器的作用则是当你知道一个样本的特征信息后想要尽可能准确地将这个样本分入正确地类别。

所以这时我们给出判断类别是否正确的标准。

贝叶斯分类器和贝叶斯决策论都是在运用概率来进行判断。所以我们的贝叶斯判别标准也同样是用概率表达的。假设给定一个样本的特征集合$F_0$={$f_1,f_2,f_3,...,f_i,...,f_n$}，类别集合$C$={$c_1,c_2,c_3,...,c_i,...,c_n$}，那么

$$
P(c_0|F_0)=max \prod_{f_i\in F_0}P(f_i|c_0)P(c_0)\qquad(6)
$$

在$(6)$，$c_0$是朴素贝叶斯分类器判断的给定样本的正确类别（但不一定就是这个样本的真实类别！）。实际上，朴素贝叶斯分类器会将所有的类别条件概率都计算一次，而朴素贝叶斯分类器就是想找到类别条件概率最大的那个类别，或者说这个样本最符合的类别。

## 贝叶斯分类器在金融的应用：运用财务指标对上市公司的质量进行判断

我国在1998年在A股市场实行了特别风险提示（Special Treatment）制度来完善我国股市在对于财务状况不佳或其他经营状况不佳的上市公司的处理制度。对于连续两个财年利润为负或者其它经营状况不佳的情况的上市公司，证监会要求这类上市公司，其股票名称要在前面冠名"ST"并单独披露信息，每日股价涨跌幅限制在±5%。

E.Altman（1986）对美国的破产清算的上市公司进行研究提出了基于财务指标的用于判断上市公司是否可能在未来陷入财务困境的Z值模型。我们可以通过python程序来编写一个朴素贝叶斯分类器来观察朴素贝叶斯分类器在判断我国上市困境的财务困境是否拥有更好的预测效果。

In [2]:
import pandas as pd
from scipy import stats

In [3]:
def get_data(name,file='.xlsx'):
    return pd.read_excel(name+file)
#我们可以用pandas导入我们的数据
#这里我们选取了2011到2012年的ST公司和与其类似的财务状况良好的上市公司
data=get_data('data')
data.head()

,证券代码,证券简称,营运资本/总资产,留存收益/总资产,息税前利润(TTM)/总资产,当日总市值/负债总计,股东权益合计(含少数)/负债总计,营业收入/总资产,Z值,类别
0,600265.SH,*ST景谷,-10.1488,-27.5182,-18.8759,192.1327,34.6317,52.0662,0.5430,ST
1,600250.SH,南纺股份,-35.7896,-6.8155,-0.5253,46.1429,5.2557,157.0272,1.3033,ST
2,600359.SH,新农开发,-22.5250,-15.3026,-18.4559,64.8052,6.7038,42.6530,-0.2786,ST
3,000972.SZ,*ST中基,-31.1206,-28.2480,-24.1481,50.4261,1.6415,32.0165,-0.9434,ST
4,600392.SH,盛和资源,50.8419,-9.8346,-16.3604,230.6682,31.7013,43.9781,1.7559,ST


In [4]:
print(len(data))#显示我们的样本总量

84


In [13]:
#编写朴素贝叶斯分类器
class naive_bayes_classifier(object):
    
    def __init__(self,features,labels,label_name,data):
        self.features = features
        self.labels = labels
        self.data = data
        self.label = label_name
    
    #计算先验概率，公式：
    def compute_prior_probability(self):
        prior_probability = {}
        for name in self.labels:
            prior_probability['name']=len(self.data[self.data[self.label]=='name'])/len(self.data)
        return prior_probability
    
    def compute_likelihood(self):
        
        def get_normal_distribution(mean,standard_diviation):
            return lambda x: stats.norm.pdf(x,mean,standard_diviation)
        
        likelihood = {}
        for feature in self.features:
            for label in self.labels:
                data = self.data.loc[:,[feature,self.label]]
                mean = data[data[self.label]==label].mean()[0]
                std = data[data[self.label]==label].std()[0]
                likelihood[feature+label]=get_normal_distibution(mean,std)
        return likelihood